In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

sns.set_style('darkgrid')
sns.set_context('notebook')

In [2]:
def check_cols(df):
    for col in df.columns:
        print('------------------------------------------------------')
        print(col)
        print(f'unique values: {df[col].nunique(dropna=False)} (\'NaN\' included)')
        print(f'NaN exists? {df[col].isna().any()}')
        if df[col].isna().any():
            print(f'how many \'NaN\' values? {df[col].isna().sum()} ({np.divide(df[col].isna().sum(),df.shape[0])*100:.2f}%)')
        print('------------------------------------------------------')
        print(df[col].value_counts(dropna=False, normalize=True))
        print('------------------------------------------------------')
        print('\n') # Break up the output between columns
    return None

In [3]:
def find_null_cols(df):
    df_clone = df.copy()
    print('\'NaN\' found in columns as follows:')
    for col in df_clone.columns:
        if (df_clone[col].isnull().any()) | (df_clone[col].isna().any()):
            print(f'\t{col:20} ({df_clone[col].dtypes})      contains {df_clone[col].isna().sum():10} null values ({df_clone[col].isna().sum()*100/df_clone.shape[0]:5.2f}%) !')
    return None

In [4]:
def fill_null_cols(df, col, nullified=False):
    df_clone = df.copy()
    if nullified:
        df_clone[col] = df_clone[col].fillna(0)
    else:
        try:
            median = df_clone[col].median()
            mean = df_clone[col].mean()
            print(f'{col}: median {median:.3f}, mean {mean:.3f}')
            if median > 0:
                df_clone[col] = df_clone[col].fillna(median)
            else:
                df_clone[col] = df_clone[col].fillna(mean)
        except:
            print('N/A')
    return df_clone

In [5]:
def replace_null_cols(df, col, obj=False, bina=False):
    df_clone = df.copy()
    if obj:
        df_clone[col] = df_clone[col].fillna('unknown')
    else:
        df_clone[col] = df_clone[col].fillna(-1.0) 
        #df_clone[col] = df_clone[col].fillna(0)
        if bina:
            df_clone[col] = df_clone[col].astype('int64')
    return df_clone

In [6]:
def replace_null_dfcols(df):
    df_clone = df.copy()
    for col in df_clone.columns:
        if (df_clone[col].isnull().any()) | (df_clone[col].isna().any()):
            df_clone[col] = df_clone[col].fillna(-1.0) 
    return df_clone

# 2015 census tract
Data for each census tract in the US, including DC and Puerto Rico.

In [7]:
df15ct = pd.read_csv("./kaggle/acs2015_census_tract_data.csv",index_col=None)
df15ct.head()

,CensusTract,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga,1948,940,1008,0.9,87.4,7.7,0.3,...,0.5,2.3,2.1,25.0,943,77.1,18.3,4.6,0.0,5.4
1,1001020200,Alabama,Autauga,2156,1059,1097,0.8,40.4,53.3,0.0,...,0.0,0.7,0.0,23.4,753,77.0,16.9,6.1,0.0,13.3
2,1001020300,Alabama,Autauga,2968,1364,1604,0.0,74.5,18.6,0.5,...,0.0,0.0,2.5,19.6,1373,64.1,23.6,12.3,0.0,6.2
3,1001020400,Alabama,Autauga,4423,2172,2251,10.5,82.8,3.7,1.6,...,0.0,2.6,1.6,25.3,1782,75.7,21.2,3.1,0.0,10.8
4,1001020500,Alabama,Autauga,10763,4922,5841,0.7,68.5,24.8,0.0,...,0.0,0.6,0.9,24.8,5037,67.1,27.6,5.3,0.0,4.2


In [8]:
df15ct.columns

Index(['CensusTract', 'State', 'County', 'TotalPop', 'Men', 'Women',
       'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [9]:
df15ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CensusTract      74001 non-null  int64  
 1   State            74001 non-null  object 
 2   County           74001 non-null  object 
 3   TotalPop         74001 non-null  int64  
 4   Men              74001 non-null  int64  
 5   Women            74001 non-null  int64  
 6   Hispanic         73311 non-null  float64
 7   White            73311 non-null  float64
 8   Black            73311 non-null  float64
 9   Native           73311 non-null  float64
 10  Asian            73311 non-null  float64
 11  Pacific          73311 non-null  float64
 12  Citizen          74001 non-null  int64  
 13  Income           72901 non-null  float64
 14  IncomeErr        72901 non-null  float64
 15  IncomePerCap     73261 non-null  float64
 16  IncomePerCapErr  73261 non-null  float64
 17  Poverty     

In [10]:
df15ct.describe()

,CensusTract,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
count,7.400100e+04,74001.000000,74001.000000,74001.000000,73311.000000,73311.000000,73311.000000,73311.000000,73311.000000,73311.000000,...,73204.000000,73204.000000,73204.000000,73052.000000,74001.000000,73194.000000,73194.000000,73194.000000,73194.000000,73199.000000
mean,2.839113e+10,4325.591465,2127.648816,2197.942649,16.862810,62.032106,13.272581,0.727726,4.588253,0.145012,...,3.123340,1.891606,4.368093,25.667357,1983.907366,78.975238,14.621566,6.233814,0.169772,9.028663
std,1.647593e+10,2129.306903,1072.332031,1095.730931,22.940695,30.684152,21.762483,4.488340,8.794003,1.037354,...,5.881237,2.596198,3.904990,6.964881,1073.429808,8.345758,7.535786,4.042990,0.458227,5.955441
min,1.001020e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.303901e+10,2891.000000,1409.000000,1461.000000,2.400000,39.400000,0.700000,0.000000,0.200000,0.000000,...,0.400000,0.400000,1.800000,20.800000,1249.000000,74.600000,9.600000,3.500000,0.000000,5.100000
50%,2.804700e+10,4063.000000,1986.000000,2066.000000,7.000000,71.400000,3.700000,0.000000,1.400000,0.000000,...,1.400000,1.100000,3.500000,25.000000,1846.000000,80.100000,13.400000,5.500000,0.000000,7.700000
75%,4.200341e+10,5442.000000,2674.000000,2774.000000,20.400000,88.300000,14.400000,0.400000,4.800000,0.000000,...,3.500000,2.500000,5.900000,29.800000,2553.000000,84.600000,18.200000,8.100000,0.000000,11.400000
max,7.215375e+10,53812.000000,27962.000000,27250.000000,100.000000,100.000000,100.000000,100.000000,91.300000,84.700000,...,100.000000,100.000000,100.000000,80.000000,24075.000000,100.000000,100.000000,100.000000,26.500000,100.000000


In [11]:
check_cols(df15ct)

------------------------------------------------------
CensusTract
unique values: 74001 ('NaN' included)
NaN exists? False
------------------------------------------------------
29189218300    0.000014
18097340109    0.000014
35001000713    0.000014
33011002200    0.000014
50027965200    0.000014
                 ...   
6073005500     0.000014
12031014332    0.000014
37051001500    0.000014
13089023422    0.000014
13135050418    0.000014
Name: CensusTract, Length: 74001, dtype: float64
------------------------------------------------------


------------------------------------------------------
State
unique values: 52 ('NaN' included)
NaN exists? False
------------------------------------------------------
California              0.108877
Texas                   0.071148
New York                0.066459
Florida                 0.057364
Pennsylvania            0.043486
Illinois                0.042202
Ohio                    0.039891
Michigan                0.038013
North Carolina     

Name: Carpool, Length: 430, dtype: float64
------------------------------------------------------


------------------------------------------------------
Transit
unique values: 833 ('NaN' included)
NaN exists? True
how many 'NaN' values? 797 (1.08%)
------------------------------------------------------
0.0     0.300685
0.5     0.025567
0.4     0.024729
0.6     0.022040
0.7     0.019878
          ...   
82.1    0.000014
87.1    0.000014
76.2    0.000014
82.2    0.000014
78.1    0.000014
Name: Transit, Length: 833, dtype: float64
------------------------------------------------------


------------------------------------------------------
Walk
unique values: 607 ('NaN' included)
NaN exists? True
how many 'NaN' values? 797 (1.08%)
------------------------------------------------------
0.0     0.192470
0.5     0.030743
0.4     0.028229
0.6     0.027892
0.7     0.026946
          ...   
39.8    0.000014
70.2    0.000014
69.7    0.000014
53.4    0.000014
37.2    0.000014
Name: Walk, Lengt

In [12]:
find_null_cols(df15ct)

'NaN' found in columns as follows:
	Hispanic             (float64)      contains        690 null values ( 0.93%) !
	White                (float64)      contains        690 null values ( 0.93%) !
	Black                (float64)      contains        690 null values ( 0.93%) !
	Native               (float64)      contains        690 null values ( 0.93%) !
	Asian                (float64)      contains        690 null values ( 0.93%) !
	Pacific              (float64)      contains        690 null values ( 0.93%) !
	Income               (float64)      contains       1100 null values ( 1.49%) !
	IncomeErr            (float64)      contains       1100 null values ( 1.49%) !
	IncomePerCap         (float64)      contains        740 null values ( 1.00%) !
	IncomePerCapErr      (float64)      contains        740 null values ( 1.00%) !
	Poverty              (float64)      contains        835 null values ( 1.13%) !
	ChildPoverty         (float64)      contains       1118 null values ( 1.51%) !
	Prof

In [13]:
df15ct_v1 = replace_null_dfcols(df15ct)
df15ct_v1.isna().any()

CensusTract        False
State              False
County             False
TotalPop           False
Men                False
Women              False
Hispanic           False
White              False
Black              False
Native             False
Asian              False
Pacific            False
Citizen            False
Income             False
IncomeErr          False
IncomePerCap       False
IncomePerCapErr    False
Poverty            False
ChildPoverty       False
Professional       False
Service            False
Office             False
Construction       False
Production         False
Drive              False
Carpool            False
Transit            False
Walk               False
OtherTransp        False
WorkAtHome         False
MeanCommute        False
Employed           False
PrivateWork        False
PublicWork         False
SelfEmployed       False
FamilyWork         False
Unemployment       False
dtype: bool

In [14]:
df15ct_v1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CensusTract,74001.0,2.839113e+10,1.647593e+10,1.001020e+09,1.303901e+10,2.804700e+10,4.200341e+10,7.215375e+10
TotalPop,74001.0,4.325591e+03,2.129307e+03,0.000000e+00,2.891000e+03,4.063000e+03,5.442000e+03,5.381200e+04
Men,74001.0,2.127649e+03,1.072332e+03,0.000000e+00,1.409000e+03,1.986000e+03,2.674000e+03,2.796200e+04
Women,74001.0,2.197943e+03,1.095731e+03,0.000000e+00,1.461000e+03,2.066000e+03,2.774000e+03,2.725000e+04
Hispanic,74001.0,1.669625e+01,2.289794e+01,-1.000000e+00,2.300000e+00,6.900000e+00,2.020000e+01,1.000000e+02
White,74001.0,6.144438e+01,3.113581e+01,-1.000000e+00,3.790000e+01,7.100000e+01,8.820000e+01,1.000000e+02
Black,74001.0,1.313950e+01,2.170418e+01,-1.000000e+00,7.000000e-01,3.600000e+00,1.420000e+01,1.000000e+02
Native,74001.0,7.116161e-01,4.470451e+00,-1.000000e+00,0.000000e+00,0.000000e+00,4.000000e-01,1.000000e+02
Asian,74001.0,4.536147e+00,8.769371e+00,-1.000000e+00,2.000000e-01,1.300000e+00,4.800000e+00,9.130000e+01
Pacific,74001.0,1.343360e-01,1.038354e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.470000e+01


# 2015 county
Data for each county or county equivalent in the US, including DC and Puerto Rico.

In [15]:
df15c = pd.read_csv("./kaggle/acs2015_county_data.csv",index_col=None)
df15c.head()

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,1003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,1005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,1007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,1009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [16]:
df15c.columns

Index(['CensusId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen', 'Income',
       'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [17]:
df15c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CensusId         3220 non-null   int64  
 1   State            3220 non-null   object 
 2   County           3220 non-null   object 
 3   TotalPop         3220 non-null   int64  
 4   Men              3220 non-null   int64  
 5   Women            3220 non-null   int64  
 6   Hispanic         3220 non-null   float64
 7   White            3220 non-null   float64
 8   Black            3220 non-null   float64
 9   Native           3220 non-null   float64
 10  Asian            3220 non-null   float64
 11  Pacific          3220 non-null   float64
 12  Citizen          3220 non-null   int64  
 13  Income           3219 non-null   float64
 14  IncomeErr        3219 non-null   float64
 15  IncomePerCap     3220 non-null   int64  
 16  IncomePerCapErr  3220 non-null   int64  
 17  Poverty       

In [18]:
df15c.describe()

,CensusId,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
count,3220.000000,3.220000e+03,3.220000e+03,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,...,3220.000000,3220.000000,3220.000000,3220.000000,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000
mean,31393.605280,9.940935e+04,4.889694e+04,5.051241e+04,11.011522,75.428789,8.665497,1.723509,1.229068,0.082733,...,3.323509,1.612733,4.631770,23.278758,4.559352e+04,74.219348,17.560870,7.931801,0.288106,8.094441
std,16292.078954,3.193055e+05,1.566813e+05,1.626620e+05,19.241380,22.932890,14.279122,7.253115,2.633079,0.734931,...,3.756096,1.670988,3.178772,5.600466,1.496995e+05,7.863188,6.510354,3.914974,0.455137,4.096114
min,1001.000000,8.500000e+01,4.200000e+01,4.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.900000,6.200000e+01,25.000000,5.800000,0.000000,0.000000,0.000000
25%,19032.500000,1.121800e+04,5.637250e+03,5.572000e+03,1.900000,64.100000,0.500000,0.100000,0.200000,0.000000,...,1.400000,0.900000,2.700000,19.500000,4.550750e+03,70.500000,13.100000,5.400000,0.100000,5.500000
50%,30024.000000,2.603500e+04,1.293200e+04,1.305700e+04,3.900000,84.100000,1.900000,0.300000,0.500000,0.000000,...,2.400000,1.300000,3.900000,23.000000,1.050800e+04,75.700000,16.200000,6.900000,0.200000,7.600000
75%,46105.500000,6.643050e+04,3.299275e+04,3.348750e+04,9.825000,93.200000,9.600000,0.600000,1.200000,0.000000,...,4.000000,1.900000,5.600000,26.800000,2.863275e+04,79.700000,20.500000,9.400000,0.300000,9.900000
max,72153.000000,1.003839e+07,4.945351e+06,5.093037e+06,99.900000,99.800000,85.900000,92.100000,41.600000,35.300000,...,71.200000,39.100000,37.200000,44.000000,4.635465e+06,88.300000,66.200000,36.600000,9.800000,36.500000


In [19]:
check_cols(df15c)

------------------------------------------------------
CensusId
unique values: 3220 ('NaN' included)
NaN exists? False
------------------------------------------------------
51199    0.000311
13107    0.000311
13131    0.000311
13127    0.000311
13123    0.000311
           ...   
28143    0.000311
24045    0.000311
28139    0.000311
24041    0.000311
46061    0.000311
Name: CensusId, Length: 3220, dtype: float64
------------------------------------------------------


------------------------------------------------------
State
unique values: 52 ('NaN' included)
NaN exists? False
------------------------------------------------------
Texas                   0.078882
Georgia                 0.049379
Virginia                0.041304
Kentucky                0.037267
Missouri                0.035714
Kansas                  0.032609
Illinois                0.031677
North Carolina          0.031056
Iowa                    0.030745
Tennessee               0.029503
Nebraska                0.0

1.0     0.064596
0.9     0.058075
1.1     0.058075
1.3     0.054658
1.2     0.053416
          ...   
10.7    0.000311
7.1     0.000311
39.1    0.000311
8.4     0.000311
10.2    0.000311
Name: OtherTransp, Length: 93, dtype: float64
------------------------------------------------------


------------------------------------------------------
WorkAtHome
unique values: 188 ('NaN' included)
NaN exists? False
------------------------------------------------------
3.7     0.027329
3.0     0.025466
3.3     0.023602
2.7     0.021739
2.5     0.021429
          ...   
14.5    0.000311
19.0    0.000311
25.0    0.000311
23.0    0.000311
16.5    0.000311
Name: WorkAtHome, Length: 188, dtype: float64
------------------------------------------------------


------------------------------------------------------
MeanCommute
unique values: 309 ('NaN' included)
NaN exists? False
------------------------------------------------------
23.2    0.010248
22.1    0.009938
24.1    0.009938
21.3    0.009938
2

In [20]:
find_null_cols(df15c)

'NaN' found in columns as follows:
	Income               (float64)      contains          1 null values ( 0.03%) !
	IncomeErr            (float64)      contains          1 null values ( 0.03%) !
	ChildPoverty         (float64)      contains          1 null values ( 0.03%) !


In [21]:
df15c_v1 = replace_null_dfcols(df15c)
df15c_v1.isna().any()

CensusId           False
State              False
County             False
TotalPop           False
Men                False
Women              False
Hispanic           False
White              False
Black              False
Native             False
Asian              False
Pacific            False
Citizen            False
Income             False
IncomeErr          False
IncomePerCap       False
IncomePerCapErr    False
Poverty            False
ChildPoverty       False
Professional       False
Service            False
Office             False
Construction       False
Production         False
Drive              False
Carpool            False
Transit            False
Walk               False
OtherTransp        False
WorkAtHome         False
MeanCommute        False
Employed           False
PrivateWork        False
PublicWork         False
SelfEmployed       False
FamilyWork         False
Unemployment       False
dtype: bool

In [22]:
df15c_v1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CensusId,3220.0,31393.605280,16292.078954,1001.0,19032.50,30024.00,46105.500,72153.0
TotalPop,3220.0,99409.345963,319305.453731,85.0,11218.00,26035.00,66430.500,10038388.0
Men,3220.0,48896.937888,156681.276856,42.0,5637.25,12932.00,32992.750,4945351.0
Women,3220.0,50512.408075,162661.952823,43.0,5572.00,13057.00,33487.500,5093037.0
Hispanic,3220.0,11.011522,19.241380,0.0,1.90,3.90,9.825,99.9
White,3220.0,75.428789,22.932890,0.0,64.10,84.10,93.200,99.8
Black,3220.0,8.665497,14.279122,0.0,0.50,1.90,9.600,85.9
Native,3220.0,1.723509,7.253115,0.0,0.10,0.30,0.600,92.1
Asian,3220.0,1.229068,2.633079,0.0,0.20,0.50,1.200,41.6
Pacific,3220.0,0.082733,0.734931,0.0,0.00,0.00,0.000,35.3


# 2017 census tract
Data for each census tract in the US, including DC and Puerto Rico.

In [23]:
df17ct = pd.read_csv("./kaggle/acs2017_census_tract_data.csv",index_col=None)
df17ct.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,...,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,...,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,...,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,...,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,...,0.8,0.3,0.7,21.0,4787,71.4,24.1,4.5,0.0,2.3


In [24]:
df17ct.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [25]:
df17ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TractId           74001 non-null  int64  
 1   State             74001 non-null  object 
 2   County            74001 non-null  object 
 3   TotalPop          74001 non-null  int64  
 4   Men               74001 non-null  int64  
 5   Women             74001 non-null  int64  
 6   Hispanic          73305 non-null  float64
 7   White             73305 non-null  float64
 8   Black             73305 non-null  float64
 9   Native            73305 non-null  float64
 10  Asian             73305 non-null  float64
 11  Pacific           73305 non-null  float64
 12  VotingAgeCitizen  74001 non-null  int64  
 13  Income            72885 non-null  float64
 14  IncomeErr         72885 non-null  float64
 15  IncomePerCap      73256 non-null  float64
 16  IncomePerCapErr   73256 non-null  float6

In [26]:
df17ct.describe()

,TractId,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
count,7.400100e+04,74001.000000,74001.000000,74001.000000,73305.000000,73305.000000,73305.00000,73305.000000,73305.000000,73305.000000,...,73200.000000,73200.000000,73200.000000,73055.000000,74001.000000,73190.000000,73190.000000,73190.000000,73190.000000,73191.000000
mean,2.839113e+10,4384.716017,2157.710707,2227.005311,17.265444,61.309043,13.28910,0.734047,4.753691,0.147341,...,3.042825,1.894605,4.661466,26.056594,2049.152052,79.494222,14.163342,6.171484,0.171164,7.246738
std,1.647593e+10,2228.936729,1120.560504,1146.240218,23.073811,30.634461,21.60118,4.554247,8.999888,1.029250,...,5.805753,2.549374,4.014940,7.124524,1138.865457,8.126383,7.328680,3.932364,0.456580,5.227624
min,1.001020e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.303901e+10,2903.000000,1416.000000,1465.000000,2.600000,38.000000,0.80000,0.000000,0.200000,0.000000,...,0.400000,0.400000,2.000000,21.100000,1276.000000,75.200000,9.300000,3.500000,0.000000,3.900000
50%,2.804700e+10,4105.000000,2007.000000,2082.000000,7.400000,70.400000,3.80000,0.000000,1.500000,0.000000,...,1.400000,1.200000,3.800000,25.400000,1895.000000,80.600000,13.000000,5.500000,0.000000,6.000000
75%,4.200341e+10,5506.000000,2707.000000,2803.000000,21.100000,87.700000,14.60000,0.400000,5.000000,0.000000,...,3.300000,2.500000,6.300000,30.300000,2635.000000,85.000000,17.600000,8.000000,0.000000,9.000000
max,7.215375e+10,65528.000000,32266.000000,33262.000000,100.000000,100.000000,100.00000,100.000000,100.000000,71.900000,...,100.000000,100.000000,100.000000,73.900000,28945.000000,100.000000,100.000000,100.000000,22.300000,100.000000


In [27]:
check_cols(df17ct)

------------------------------------------------------
TractId
unique values: 74001 ('NaN' included)
NaN exists? False
------------------------------------------------------
29189218300    0.000014
18097340109    0.000014
35001000713    0.000014
33011002200    0.000014
50027965200    0.000014
                 ...   
6073005500     0.000014
12031014332    0.000014
37051001500    0.000014
13089023422    0.000014
13135050418    0.000014
Name: TractId, Length: 74001, dtype: float64
------------------------------------------------------


------------------------------------------------------
State
unique values: 52 ('NaN' included)
NaN exists? False
------------------------------------------------------
California              0.108877
Texas                   0.071148
New York                0.066459
Florida                 0.057364
Pennsylvania            0.043486
Illinois                0.042202
Ohio                    0.039891
Michigan                0.038013
North Carolina          0.0

how many 'NaN' values? 801 (1.08%)
------------------------------------------------------
NaN     0.010824
6.6     0.010081
8.0     0.009905
9.1     0.009284
7.3     0.009257
          ...   
58.9    0.000014
44.9    0.000014
38.8    0.000014
45.7    0.000014
59.1    0.000014
Name: Carpool, Length: 418, dtype: float64
------------------------------------------------------


------------------------------------------------------
Transit
unique values: 835 ('NaN' included)
NaN exists? True
how many 'NaN' values? 801 (1.08%)
------------------------------------------------------
0.0     0.310401
0.5     0.024297
0.4     0.023229
0.6     0.022621
0.3     0.019919
          ...   
54.2    0.000014
77.3    0.000014
83.2    0.000014
80.6    0.000014
84.6    0.000014
Name: Transit, Length: 835, dtype: float64
------------------------------------------------------


------------------------------------------------------
Walk
unique values: 597 ('NaN' included)
NaN exists? True
how many 'NaN' va

In [28]:
find_null_cols(df17ct)

'NaN' found in columns as follows:
	Hispanic             (float64)      contains        696 null values ( 0.94%) !
	White                (float64)      contains        696 null values ( 0.94%) !
	Black                (float64)      contains        696 null values ( 0.94%) !
	Native               (float64)      contains        696 null values ( 0.94%) !
	Asian                (float64)      contains        696 null values ( 0.94%) !
	Pacific              (float64)      contains        696 null values ( 0.94%) !
	Income               (float64)      contains       1116 null values ( 1.51%) !
	IncomeErr            (float64)      contains       1116 null values ( 1.51%) !
	IncomePerCap         (float64)      contains        745 null values ( 1.01%) !
	IncomePerCapErr      (float64)      contains        745 null values ( 1.01%) !
	Poverty              (float64)      contains        842 null values ( 1.14%) !
	ChildPoverty         (float64)      contains       1110 null values ( 1.50%) !
	Prof

In [29]:
df17ct_v1 = replace_null_dfcols(df17ct)
df17ct_v1.isna().any()

TractId             False
State               False
County              False
TotalPop            False
Men                 False
Women               False
Hispanic            False
White               False
Black               False
Native              False
Asian               False
Pacific             False
VotingAgeCitizen    False
Income              False
IncomeErr           False
IncomePerCap        False
IncomePerCapErr     False
Poverty             False
ChildPoverty        False
Professional        False
Service             False
Office              False
Construction        False
Production          False
Drive               False
Carpool             False
Transit             False
Walk                False
OtherTransp         False
WorkAtHome          False
MeanCommute         False
Employed            False
PrivateWork         False
PublicWork          False
SelfEmployed        False
FamilyWork          False
Unemployment        False
dtype: bool

In [30]:
df17ct_v1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TractId,74001.0,2.839113e+10,1.647593e+10,1.001020e+09,1.303901e+10,2.804700e+10,4.200341e+10,7.215375e+10
TotalPop,74001.0,4.384716e+03,2.228937e+03,0.000000e+00,2.903000e+03,4.105000e+03,5.506000e+03,6.552800e+04
Men,74001.0,2.157711e+03,1.120561e+03,0.000000e+00,1.416000e+03,2.007000e+03,2.707000e+03,3.226600e+04
Women,74001.0,2.227005e+03,1.146240e+03,0.000000e+00,1.465000e+03,2.082000e+03,2.803000e+03,3.326200e+04
Hispanic,74001.0,1.709365e+01,2.303262e+01,-1.000000e+00,2.500000e+00,7.300000e+00,2.090000e+01,1.000000e+02
White,74001.0,6.072301e+01,3.107758e+01,-1.000000e+00,3.670000e+01,7.000000e+01,8.760000e+01,1.000000e+02
Black,74001.0,1.315471e+01,2.154355e+01,-1.000000e+00,7.000000e-01,3.700000e+00,1.440000e+01,1.000000e+02
Native,74001.0,7.177376e-01,4.535868e+00,-1.000000e+00,0.000000e+00,0.000000e+00,4.000000e-01,1.000000e+02
Asian,74001.0,4.699576e+00,8.974665e+00,-1.000000e+00,2.000000e-01,1.400000e+00,5.000000e+00,1.000000e+02
Pacific,74001.0,1.365495e-01,1.030367e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.190000e+01


# 2017 county
Data for each county or county equivalent in the US, including DC and Puerto Rico.

In [31]:
df17c = pd.read_csv("./kaggle/acs2015_county_data.csv",index_col=None)
df17c.head()

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,1003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,1005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,1007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,1009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [32]:
df17c.columns

Index(['CensusId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'Citizen', 'Income',
       'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')

In [33]:
df17c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CensusId         3220 non-null   int64  
 1   State            3220 non-null   object 
 2   County           3220 non-null   object 
 3   TotalPop         3220 non-null   int64  
 4   Men              3220 non-null   int64  
 5   Women            3220 non-null   int64  
 6   Hispanic         3220 non-null   float64
 7   White            3220 non-null   float64
 8   Black            3220 non-null   float64
 9   Native           3220 non-null   float64
 10  Asian            3220 non-null   float64
 11  Pacific          3220 non-null   float64
 12  Citizen          3220 non-null   int64  
 13  Income           3219 non-null   float64
 14  IncomeErr        3219 non-null   float64
 15  IncomePerCap     3220 non-null   int64  
 16  IncomePerCapErr  3220 non-null   int64  
 17  Poverty       

In [34]:
df17c.describe()

,CensusId,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
count,3220.000000,3.220000e+03,3.220000e+03,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,...,3220.000000,3220.000000,3220.000000,3220.000000,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000
mean,31393.605280,9.940935e+04,4.889694e+04,5.051241e+04,11.011522,75.428789,8.665497,1.723509,1.229068,0.082733,...,3.323509,1.612733,4.631770,23.278758,4.559352e+04,74.219348,17.560870,7.931801,0.288106,8.094441
std,16292.078954,3.193055e+05,1.566813e+05,1.626620e+05,19.241380,22.932890,14.279122,7.253115,2.633079,0.734931,...,3.756096,1.670988,3.178772,5.600466,1.496995e+05,7.863188,6.510354,3.914974,0.455137,4.096114
min,1001.000000,8.500000e+01,4.200000e+01,4.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.900000,6.200000e+01,25.000000,5.800000,0.000000,0.000000,0.000000
25%,19032.500000,1.121800e+04,5.637250e+03,5.572000e+03,1.900000,64.100000,0.500000,0.100000,0.200000,0.000000,...,1.400000,0.900000,2.700000,19.500000,4.550750e+03,70.500000,13.100000,5.400000,0.100000,5.500000
50%,30024.000000,2.603500e+04,1.293200e+04,1.305700e+04,3.900000,84.100000,1.900000,0.300000,0.500000,0.000000,...,2.400000,1.300000,3.900000,23.000000,1.050800e+04,75.700000,16.200000,6.900000,0.200000,7.600000
75%,46105.500000,6.643050e+04,3.299275e+04,3.348750e+04,9.825000,93.200000,9.600000,0.600000,1.200000,0.000000,...,4.000000,1.900000,5.600000,26.800000,2.863275e+04,79.700000,20.500000,9.400000,0.300000,9.900000
max,72153.000000,1.003839e+07,4.945351e+06,5.093037e+06,99.900000,99.800000,85.900000,92.100000,41.600000,35.300000,...,71.200000,39.100000,37.200000,44.000000,4.635465e+06,88.300000,66.200000,36.600000,9.800000,36.500000


In [35]:
check_cols(df17c)

------------------------------------------------------
CensusId
unique values: 3220 ('NaN' included)
NaN exists? False
------------------------------------------------------
51199    0.000311
13107    0.000311
13131    0.000311
13127    0.000311
13123    0.000311
           ...   
28143    0.000311
24045    0.000311
28139    0.000311
24041    0.000311
46061    0.000311
Name: CensusId, Length: 3220, dtype: float64
------------------------------------------------------


------------------------------------------------------
State
unique values: 52 ('NaN' included)
NaN exists? False
------------------------------------------------------
Texas                   0.078882
Georgia                 0.049379
Virginia                0.041304
Kentucky                0.037267
Missouri                0.035714
Kansas                  0.032609
Illinois                0.031677
North Carolina          0.031056
Iowa                    0.030745
Tennessee               0.029503
Nebraska                0.0

Name: Unemployment, Length: 230, dtype: float64
------------------------------------------------------




In [36]:
find_null_cols(df17c)

'NaN' found in columns as follows:
	Income               (float64)      contains          1 null values ( 0.03%) !
	IncomeErr            (float64)      contains          1 null values ( 0.03%) !
	ChildPoverty         (float64)      contains          1 null values ( 0.03%) !


In [37]:
df17c_v1 = replace_null_dfcols(df17c)
df17c_v1.isna().any()

CensusId           False
State              False
County             False
TotalPop           False
Men                False
Women              False
Hispanic           False
White              False
Black              False
Native             False
Asian              False
Pacific            False
Citizen            False
Income             False
IncomeErr          False
IncomePerCap       False
IncomePerCapErr    False
Poverty            False
ChildPoverty       False
Professional       False
Service            False
Office             False
Construction       False
Production         False
Drive              False
Carpool            False
Transit            False
Walk               False
OtherTransp        False
WorkAtHome         False
MeanCommute        False
Employed           False
PrivateWork        False
PublicWork         False
SelfEmployed       False
FamilyWork         False
Unemployment       False
dtype: bool

In [38]:
df17c_v1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CensusId,3220.0,31393.605280,16292.078954,1001.0,19032.50,30024.00,46105.500,72153.0
TotalPop,3220.0,99409.345963,319305.453731,85.0,11218.00,26035.00,66430.500,10038388.0
Men,3220.0,48896.937888,156681.276856,42.0,5637.25,12932.00,32992.750,4945351.0
Women,3220.0,50512.408075,162661.952823,43.0,5572.00,13057.00,33487.500,5093037.0
Hispanic,3220.0,11.011522,19.241380,0.0,1.90,3.90,9.825,99.9
White,3220.0,75.428789,22.932890,0.0,64.10,84.10,93.200,99.8
Black,3220.0,8.665497,14.279122,0.0,0.50,1.90,9.600,85.9
Native,3220.0,1.723509,7.253115,0.0,0.10,0.30,0.600,92.1
Asian,3220.0,1.229068,2.633079,0.0,0.20,0.50,1.200,41.6
Pacific,3220.0,0.082733,0.734931,0.0,0.00,0.00,0.000,35.3
